In [4]:
try:
    spark.stop()
except Exception:
    pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Iceberg via REST")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local.type", "rest")
    .config("spark.sql.catalog.local.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.local.warehouse", "s3://lake/warehouse")
    .config("spark.sql.catalog.local.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.local.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.local.s3.path-style-access", "true")
    .config("spark.sql.catalog.local.s3.access-key-id", "admin")
    .config("spark.sql.catalog.local.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.local.s3.region", "us-east-1")
    .getOrCreate()
)

spark

In [5]:
spark.sql("SHOW NAMESPACES IN local").show(truncate=False)

+---------+
|namespace|
+---------+
|raw      |
+---------+



In [7]:
spark.sql("SHOW TABLES IN local.raw").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|raw      |avito    |false      |
+---------+---------+-----------+



In [11]:
tbl = "local.raw.avito"
spark.sql(f"SELECT COUNT(*) AS rows FROM {tbl}").show()


+----+
|rows|
+----+
| 160|
+----+



In [ ]:
spark.sql("SELECT * FROM local.raw.avito LIMIT 5").show(truncate=False)


In [13]:
# Load RAW table
raw_df = spark.table("local.raw.avito")   # or local.raw.sarouty

In [16]:
# Créer un DataFrame "Silver" minimal
silver_df = raw_df.select("id").distinct()

# Afficher quelques lignes
silver_df.show(5, truncate=False)

print("✅ Silver dataset initialized with only 'id' column.")
print("Total IDs:", silver_df.count())

+--------+
|id      |
+--------+
|55888606|
|55853042|
|57133328|
|57020711|
|57130151|
+--------+
only showing top 5 rows

✅ Silver dataset initialized with only 'id' column.
Total IDs: 135
